*Load Required Libraries*

In [1]:
# Libraries for data processing
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
from pprint import pprint
import tqdm
import datetime

# NLP set-up 
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
lemmaObj = WordNetLemmatizer()

# Libraries for Word Cloud
from wordcloud import WordCloud
from tqdm import tqdm
from collections import Counter

# Libraries for data processing and extraction
from sklearn import decomposition
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# Libraries for visualization
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
%matplotlib inline

# Libraries for Topic Modelling 
import gensim
from gensim.models import LdaModel, CoherenceModel, LsiModel, HdpModel
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tanay\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\tanay\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
C:\Users\tanay\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
C:\Users\tanay\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\gensim\matutils.py:22: DeprecationWarning: Please use `triu` from the `scipy.linalg` namespace, the `scipy.linalg.special_matrices` namespace is deprecated.
  from scipy.linalg.special_matrices im

In [2]:
df = pd.read_csv('/Users/tanay/Downloads/finalprocessed_data.csv')

In [3]:
df

,Name,Comment,Date,Model,sentiment_score,Source,cleaned_review,sentiment_label
0,Gumpclipper,"Good morning all,I am looking at buying my fir...",2021-10-10,Porsche Macan,5,Pistonheads,good morning looking buying advice essential ...,positive
1,Voodoo Blue,I’m not going to offer an opinion on spec as t...,2021-10-10,Porsche Macan,4,Pistonheads,going offer opinion spec pretty subjective sug...,positive
2,Gumpclipper,Fantastic info. Thank you! Will take a look at...,2021-10-10,Porsche Macan,5,Pistonheads,fantastic info look forum cheer,positive
3,TheBMWDriver,Right now 1% is best you can get on the macan....,2021-10-21,Porsche Macan,4,Pistonheads,right percent best speci find k option make ...,positive
4,neilf,My tip would be to get your name on the deposi...,2021-10-22,Porsche Macan,1,Pistonheads,tip deposit list opc asap way queue decision d...,negative
...,...,...,...,...,...,...,...,...
48940,Binkyjean,The width of the Levante and Grecale are almos...,2022-03-25,Maserati Grecale,2,Forums,width levante inch mirror,negative
48941,Mii,"1,948 mm (76.7 in) is listed on Wiki, should b...",2022-04-02,Maserati Grecale,3,Forums,mm listed wiki width mirror folded http en...,neutral
48942,Harvey Wiggins,"Finally, I found a spec that listed body width...",2022-04-02,Maserati Grecale,3,Forums,finally found spec listed body width mirror wi...,neutral
48943,Mii,"Need to see the gear button control in person,...",2022-04-02,Maserati Grecale,3,Forums,gear button control person non gear lever pref...,neutral


In [4]:
# Filter rows with Model "Porsche Macan"
x3_data = df[df['Model'] == 'BMW X3']

x3_data

,Name,Comment,Date,Model,sentiment_score,Source,cleaned_review,sentiment_label
25726,arbit,"Hi,I am looking to replace my current BMW E92 ...",2021-03-27,BMW X3,4,Pistonheads,hi looking replace current e i mi arrival ba...,positive
25727,RBrown,Hi. I have March 2018 X3 M40i. It was one of t...,2021-03-27,BMW X3,4,Pistonheads,hi march mi uk love great family wagon give ...,positive
25728,cossy400,Sorry im in for this as we are planning on get...,2021-03-27,BMW X3,1,Pistonheads,sorry im planning getting rid soon lock d one...,negative
25729,RBrown,My wife has previous generation (F25) X3 20d. ...,2021-03-27,BMW X3,4,Pistonheads,wife previous generation f d far prefers mi s...,positive
25730,Cobnapint,I'm amazed (and pleased) to hear the ride is s...,2021-03-29,BMW X3,4,Pistonheads,amazed pleased hear ride smooth mi lead belie...,positive
...,...,...,...,...,...,...,...,...
37797,Doctor Jeff,"I'm warming up to the X4 looks, but I'm a litt...",2016-04-06,BMW X3,3,Forums,warming look little confused performance ve...,neutral
37798,BobinIl,Doctor Jeff said:I'm warming up to the X4 look...,2016-04-06,BMW X3,3,Forums,doctor jeff warming look little confused pe...,neutral
37799,Doctor Jeff,"The XC90 is pretty awesome IMO, just too big f...",2016-04-06,BMW X3,3,Forums,xc pretty awesome imo big purpose cheap looked...,neutral
37800,Bernie@Bimmerfest,BMW's next gen X3 (G01) was spied during some ...,2016-03-23,BMW X3,3,Forums,gen g spied snow testing recently wearing c...,neutral


*LDA Model*

In [5]:
# Create Dictionary
dicton = corpora.Dictionary(x3_data['cleaned_review'].str.split())

# Create Corpus
texts = x3_data['cleaned_review'].str.split()

# Term Document Frequency
corpus = [dicton.doc2bow(text) for text in texts]

print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1), (20, 1), (21, 1), (22, 2), (23, 2), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)]


In [6]:
# Build a LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=dicton,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [7]:
# Print the keywords in topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.015*"turn" + 0.014*"light" + 0.013*"like" + 0.012*"brake" + 0.010*"stop" '
  '+ 0.010*"start" + 0.010*"button" + 0.008*"driving" + 0.007*"auto" + '
  '0.007*"speed"'),
 (1,
  '0.078*"tire" + 0.041*"wheel" + 0.019*"spare" + 0.018*"run" + 0.018*"flat" + '
  '0.017*"winter" + 0.013*"set" + 0.012*"non" + 0.010*"hitch" + '
  '0.010*"season"'),
 (2,
  '0.022*"mi" + 0.017*"like" + 0.017*"sport" + 0.015*"look" + 0.013*"i" + '
  '0.011*"sent" + 0.009*"iphone" + 0.007*"model" + 0.007*"wheel" + '
  '0.007*"package"'),
 (3,
  '0.020*"issue" + 0.016*"problem" + 0.016*"thanks" + 0.012*"dealer" + '
  '0.012*"warranty" + 0.009*"battery" + 0.008*"usb" + 0.008*"fob" + '
  '0.007*"owner" + 0.007*"fix"'),
 (4,
  '0.021*"door" + 0.017*"sound" + 0.012*"digital" + 0.011*"mirror" + '
  '0.011*"key" + 0.010*"display" + 0.009*"window" + 0.008*"garage" + '
  '0.008*"setting" + 0.007*"like"'),
 (5,
  '0.054*"dollar" + 0.028*"dealer" + 0.019*"com" + 0.018*"k" + 0.018*"price" + '
  '0.018*"percent" + 0.01

In [8]:
# Calculate the overall Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dicton, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.5398215605607273


In [9]:
# Supporting Model 
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dicton, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [10]:
grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 12
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=dicton, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results)
    model_results
    pbar.close()

100%|██████████| 600/600 [3:20:33<00:00, 20.06s/it]  


In [11]:
x3_results= pd.DataFrame(model_results)
x3_results

,Validation_Set,Topics,Alpha,Beta,Coherence
0,75% Corpus,2,0.01,0.01,0.599492
1,75% Corpus,2,0.01,0.31,0.620349
2,75% Corpus,2,0.01,0.61,0.583401
3,75% Corpus,2,0.01,0.91,0.630792
4,75% Corpus,2,0.01,symmetric,0.620349
...,...,...,...,...,...
595,100% Corpus,11,asymmetric,0.01,0.515671
596,100% Corpus,11,asymmetric,0.31,0.501503
597,100% Corpus,11,asymmetric,0.61,0.507935
598,100% Corpus,11,asymmetric,0.91,0.544663


In [12]:
x3_results.to_csv('/Users/tanay/Downloads/x3_cohenrencef.csv', index=False, encoding="utf-8-sig")

In [21]:
# Number of Topics 4 has the highest coherence score (0.63), therefore we continue with that
num_topics = 4

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dicton,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=15,
                                           alpha=0.02,
                                           eta=0.9)

In [22]:
# Print the topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.041*"tire" + 0.016*"wheel" + 0.010*"spare" + 0.009*"winter" + '
  '0.008*"flat" + 0.008*"run" + 0.006*"set" + 0.005*"rear" + 0.005*"snow" + '
  '0.005*"season"'),
 (1,
  '0.022*"dollar" + 0.016*"dealer" + 0.010*"k" + 0.008*"com" + 0.007*"order" + '
  '0.007*"http" + 0.007*"percent" + 0.007*"price" + 0.007*"www" + '
  '0.006*"production"'),
 (2,
  '0.013*"like" + 0.011*"mi" + 0.010*"sport" + 0.010*"seat" + 0.010*"look" + '
  '0.008*"i" + 0.008*"wheel" + 0.006*"package" + 0.005*"sent" + '
  '0.005*"driving"'),
 (3,
  '0.007*"issue" + 0.005*"work" + 0.005*"like" + 0.005*"problem" + '
  '0.005*"dealer" + 0.005*"door" + 0.004*"vehicle" + 0.004*"start" + '
  '0.004*"use" + 0.004*"thanks"')]


In [23]:
# Visualize the topics
pyLDAvis.enable_notebook()
if 1 == 1:
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, dicton)

LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.026193 -0.072952       1        1  34.050788
3     -0.031266 -0.072499       2        1  31.865836
1     -0.087394  0.108707       3        1  22.819585
0      0.144854  0.036745       4        1  11.263791, topic_info=        Term         Freq        Total Category  logprob  loglift
2047    tire  1216.000000  1216.000000  Default  30.0000  30.0000
2101  dollar  1408.000000  1408.000000  Default  29.0000  29.0000
99     wheel  1155.000000  1155.000000  Default  28.0000  28.0000
10    dealer  1422.000000  1422.000000  Default  27.0000  27.0000
143    sport   921.000000   921.000000  Default  26.0000  26.0000
...      ...          ...          ...      ...      ...      ...
20      like    86.931207  1960.124121   Topic4  -5.8224  -0.9321
1115     use    72.111424   553.768408   Topic4  -6.0093   0.1450
23        mi    74.858757  1545.516857   Topic4  -5.9719  -0.8439
1499       g    69.711578   531.294677   Topic4  -6.0431   0.1526
34      ride    66.740577   307.665622   Topic4  -6.0867   0.6554

[269 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1         1  0.992734  adaptive
1         2  0.003064  adaptive
1         3  0.003064  adaptive
1         4  0.003064  adaptive
2906      1  0.024855     alarm
...     ...       ...       ...
258       4  0.022531       www
1308      1  0.978181        xc
1308      2  0.009406        xc
1308      3  0.009406        xc
1308      4  0.009406        xc

[896 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 2, 1])